# Estimating the Borrelia burgdorferi strain diversity in North-American tick samples
## Cedric Chauve, December 20, 2019

In this notebook, I describe a novel method to estimate the diversity of a bacterial pathogen, namely *Borrelia burgdorferi*, from high-throughput sequencing data collected from 24 North-American tick samples. The goal is twofold -- introduce a novel method and illustrate it on real data -- with the aim to both improve the method and design a realstic simulation pipeline allowing to estimate its accuracy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Directories
DIR_DATA = '../data'
DIR_RES  = '../results'
# Samples list
SAMPLES = [s.rstrip() for s in open(DIR_DATA+'/samples_list.txt','r').readlines()]

## Data and preprocessing

### MLST
We estimate the diversity of *Borrelia* strains in each sample using the classical MLST scheme available from https://pubmlst.org/borrelia/, downloaded on October 11, 2019. This scheme contains 8 genes, *clpA, clpX, nifS, pepX, pyrG, recG, rplB, uvrA*. For each of these genes, the available alleles were aligned using MAFFT to form a multiple sequence alignment (MSA). The alleles sequences for gene **X** are available in the files '../data/X.fasta' and the MSA in the file '../data/X_mafft.fasta'.

In [15]:
# MLST analysis.
LOCI = ['clpA', 'clpX', 'nifS', 'pepX', 'pyrG', 'recG', 'rplB', 'uvrA']

for locus in LOCI:
    nb_alleles = int((len(open(DIR_DATA+'/'+locus+'.fasta','r').readlines())-1)/2)
    msa_lg     = int(len(open(DIR_DATA+'/'+locus+'_mafft.fasta','r').readlines()[2]))
    print(locus+' has '+str(nb_alleles)+' alleles and its MSA has '+str(msa_lg)+' columns')

MLST scheme statistics

clpA has 301 alleles and its MSA has 592 columns
clpX has 261 alleles and its MSA has 628 columns
nifS has 235 alleles and its MSA has 565 columns
pepX has 264 alleles and its MSA has 571 columns
pyrG has 277 alleles and its MSA has 607 columns
recG has 292 alleles and its MSA has 652 columns
rplB has 255 alleles and its MSA has 625 columns
uvrA has 268 alleles and its MSA has 580 columns


### Known strains
We downloaded also from https://pubmlst.org/borrelia/ a list of knwon strain types (ST), each being composed of a set of 8 alleles, one per locus, and originate from North America, Asia and Europe. They are available in the file '../bigsdb.txt'.

In [16]:
# ST database analysis

nb_ST = int((len(open(DIR_DATA+'/bigsdb.txt','r').readlines())-1))
print('There are '+str(nb_ST)+' known STs.')

There are 927 known STs.


### ST preprocessing

In order to map sequence data onto the MLST data, we created a single DNA sequence per known ST as follows. We first downloaded a fully assembled *Borrelia burgdorferi* strain, the strain *B331* (https://www.ncbi.nlm.nih.gov/nuccore/CP017201). Then for each of the 8 loci of the MLST scheme, we aligned, using BLASTn, all alleles onto this strain and recorded the location of the best alignment, correspondign to the allele present in the strain. Then we extracted from the genome 75bp before and after each such alignment (called the *flanking regions* from now). Finally for each ST, for each locus, we flanked the allele present in the ST by the corresponding flanking regions and concatenated all these sequences into a single sequence by a segment of 30 Ns. We call these sequences the *flanked STs*.

### Samples preprocessing: mapping reads

For each of the 24 samples, we have a set of paired-end reads of 76bp (so very short). The depth of sequencing of the various samples varies a lot, as the number of reads per sample goes from 272,573 (SRR2034333) to 26,870,078 (SRR2034336).

For each sample, we considered the reads a single-end reads, i.e. we did not use the pairing information. We mapped the sample reads onto the 927 sequences defined as above (one for each known ST) using bowtie with the following command:

bowtie2 -f -a --local --very-sensitive

Then for each read, we conserved a single mapping considered as the best mapping. For a given read, we expect it will align well within the same locus of many flanked STs. So we kept as best mapping the one with the maximum number of matched bases withiin the allele (i.e. not including the flanking regions). For reads overlapping both the allele and a flanking region (prefix or suffix), we kept the mapping only if at least 38bp (i.e. half of the read) mapped within the allele. Last, for each kept mapped read, we translated the mapping into a mapping onto the MSA for this allele, excluding mapping that would define an insertion (i.e. extend the length of the MSA). This results in a subset of reads, each associated to a specific locus of the MLST scheme, and for this locus, to a set of positions (columns) of the MSA with a nucleotide value per position (A,C,G,T or X if the best mapping indicates a deletion in the read).

For each sample **X**, the results of this mapping phase can be found in two files, '../results/X_bt2_nucpos.txt' that gives the nucleotides observed at each position of the MSA of each locus, and '../results/X_bt2_phasing_aux.txt' giving for each kept read the positions of the MSA it maps to and the nucleotide observed in the read at this position. 

### Samples preprocessing: identifying variable positions and possible STs

Next, for each sample, we filter out some positions of the MSA and some observed nucleotides according to the following criteria, considered in this order:
- we compute, for each locus, the average coverage of the locus by mapped reads and the standard deviation of this coverage,
- we filter out every position whose coverage is smaller than the mean for the locus by at least 2 standard deviations,
- we filter out every nucleotide observed a single time or with frequency at most 1% at a given position (assumed to be a sequencing error)
An unfiltered  position with more than one observed nucleotide (or gap X) is then called a *variable position* indicating it contains diversity signal, while a position with a single observed nucleotide is an *invariant position*.

Last, we consider the STs in the database of known STs and we consider that an ST is possible in a given sample if its alleles are fully covered (with no condition of depth of coverage) by the mapped reads. Once possible STs have been identified, we group them into equivalence classes in terms of their genotype when limited to variable positions.

|Sample|Sequenced reads|Mapped reads|Filtered out MSA positions|Filtered out nucleotides|Variable positions|Possible STs eq. classes|
| --- | --- | --- | --- | --- | --- | --- |
|SRR2034333|272572     |1195  |30 |0   |6  |1|
|SRR2034340|350468     |1578  |90 |0   |13 |0|
|SRR2034350|1.22076e+06|1853  |62 |0   |25 |0|
|SRR2034349|443552     |2001  |78 |0   |9  |0|
|SRR2034355|541816     |3067  |104|0   |34 |0|
|SRR2034346|636620     |3158  |39 |0   |26 |0|
|SRR2034356|725966     |4309  |129|0   |40 |1|
|SRR2034339|695812     |4512  |129|0   |37 |2|
|SRR2034334|942236     |5777  |106|0   |49 |4|
|SRR2034335|892872     |5892  |131|0   |48 |2|
|SRR2034347|1.0617e+06 |7430  |167|0   |57 |1|
|SRR2034351|1.30208e+06|8610  |156|0   |51 |1|
|SRR2034345|1.39423e+06|8937  |123|1   |53 |1|
|SRR2034362|1.56626e+06|9530  |189|0   |45 |1|
|SRR2034353|1.58816e+06|9898  |180|10  |80 |1|
|SRR2034343|1.85556e+06|12611 |185|43  |91 |2|
|SRR2034341|2.2334e+06 |16298 |192|69  |30 |1|
|SRR2034360|3.01241e+06|21166 |195|115 |27 |1|
|SRR2034338|2.92979e+06|22653 |182|118 |28 |1|
|SRR2034348|3.25138e+06|24461 |177|145 |48 |5|
|SRR2034342|4.5508e+06 |32418 |220|193 |26 |1|
|SRR2034344|5.99737e+06|35374 |199|197 |24 |1|
|SRR2034352|1.51766e+07|102777|208|1304|26 |1|
|SRR2034336|2.68701e+07|117265|168|979 |30 |1|

#### Comments
The most important comment is that the size of the problem, after all these preprocessing steps, is generally small. The number of variable positions does not exceed 91 and the number of possible known STs that could be found in the sample is very low (maximum of 5). This last observation is very surprising. 

Given te three lowest values of the number of variable positions occur for samples with a low depth of coverage one can however think that there might be an issue here with low abundance STs that might not have been serquenced.

A question that is also immediate when looking at these statistics is the following: can-we determine, or at least guess, from these data if there is only within-host evolution of co-infection?

#### Remark
This part of the work is the one I am the less comfortable with. The flanking of the alleles is a hack, the way to define if a read maps well on a suffix or a prefix of an allele is quite arbitrary, ... 

For example, looking at the beginning and end of alleles, I can see that they are much less covered than the middle of the alleles. For example, for the locus clpA of sample SRR2034362, we observe in '../results/SRR2034362_bt2_nucpos.txt'

clpA_0	A:61 C:0 G:0 T:0 X:0  
clpA_1	A:62 C:0 G:0 T:1 X:0  
clpA_2	A:64 C:0 G:0 T:0 X:0  
...  
clpA_330	A:0 C:0 G:0 T:148 X:0   
clpA_331	A:0 C:0 G:0 T:148 X:0   
clpA_332	A:0 C:0 G:0 T:151 X:0   
...  
clpA_588	A:0 C:0 G:0 T:12 X:71   
clpA_589	A:0 C:0 G:0 T:80 X:0   
clpA_590	A:0 C:0 G:0 T:76 X:0   


I think there is a lot of room for improvement here, for example using a colored de Bruijn graph to identify. The work presented by Kevin Da Silva at SeqBIM could be helpful.

## Method

Now I describe the method we designed to infer the diversity of STs in a sample. The method works sample per sample, and does not consider several samples at once. 

Assuming we have an instance with $N$ variable positions, a solution with $K$ STs is defined a three elements:
- a vector $A$ of $K$ positive real numbers summing up to $1$, representing the relative abundances of the $K$ STs; 
- a $K\times N$ matrix $G$ whose entries at column $j$ belong to the nucleotides (a subset of {A,C,G,T,X}) observed at the $j^{th}$ variable position; row $i$ of this matrix represents the genotype of ST $i$;
- a vector $E$ that classifies every ST to be either a known ST of the filtered database or a novel ST.

The cost of a solution is defined by a function with 4 terms.

##### Novel ST usage.
The novel ST usage is the sum of the abundances of the inferred STs that are classified as novel. we denote it by $NSU$.

##### Maximum and average nucleotide frequency deviation.
For a variable position $j$ and a nucleotide observed at this position in the data, say A, we define the *deviation* for A at $j$ as the absolute value of the difference between the observed frequency of $A$ (an input to the problem) and the frequency of $A$ defined by the inferred STs (the sum of the abundances of the inferred STs having $A$ a position $j$). The sum of the deviations for all nucleotides observed at position $j$ is denoted by $DEV(j)$. 

The average (resp. maximum) deviation for a solution is the sum (resp. maximum) of $DEV(j)$ over all variable positions $j$ divided by $N$ the number of variable positions, and is denoted by $DEV\_AVG$ (resp. $DEV\_MAX$).

##### Phasing ratio
Last, we consider all mapped reads. Each such read maps over a set of at least one variable position, and we consider only reads mapping over at least two variable positions. Such a read is said to be *phased* if there exists at least one inferred ST for which the content at these positions is the content observed in the read. The *phasing ratio* is the ratio of the number of non-phased reads over the number reads mapping over several variable positions. We denote it by $UPR=UNPHASED\_READS\_RATIO$. Note that this approach does not use the reads pairong information but it is easy to incorporate (experiments with this extension are running).

##### Objective function
It aims to minimize $$NSU+DEV\_AVG+DEV\_MAX+UPR$$

We solve it using an ILP implemented in CPLEX where we fix the parameter $K$ (it is the only parameter).

## Results

For every sample, we ran this method for $K=1,2,3,4,5,6$. Fr sample X results are available in the files
- '../results/X_bt2_2_1_01_S_5_XMIN_00_phasing.log' (CPLEX log file)
- '../results/X_bt2_2_1_01_S_5_XMIN_00_phasing_sol.txt' (solution summary)

To visualize all the reslts for sample X, it is sufficient to type in a python cell of this notebook the command 'show_res(X)'

In [58]:
def show_res(X):
    print('Sample '+X)
    K = [1,2,3,4,5,6]
    for k in K:
        print('--- K='+str(k)+'------------------------------------------------------------------')
        for l in open('../results/X_bt2_2_1_01_S_'.replace('X_',X+'_')+str(k)+'_XMIN_00_phasing_sol.txt','r').readlines():
            if l[0]!='#':
                print('\t'+l.rstrip().replace('Dev\t','Deviation').replace('Strain usage penalty','NSU\t').replace('Normalized cumulated deviation penalty','DEV_AVG\t').replace('Maximum deviation penalty','DEV_MAX\t').replace('Ratio of phased reads','UNPHASED'))

In [59]:
show_res('SRR2034336')

Sample SRR2034336
--- K=1------------------------------------------------------------------
	 	    	     	1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
	0	1.0	Novel	T  A  T  A  C  A  A  A  G  G  G  A  T  A  X  A  T  G  G  C  C  G  A  T  T  A  C  X  A  G
	Deviation		03 56 22 15 09 17 03 03 1  05 03 03 05 03 29 04 06 1  06 03 08 02 04 03 04 03 02 25 03 03
	Objective function: 1.804234045234414
	NSU	: 1.0
	DEV_AVG	: 0.09012218945708214
	DEV_MAX	: 0.5635910224439988
	UNPHASED: 0.15052083333333333
--- K=2------------------------------------------------------------------
	 	    	     	1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
	0	0.41	3	T  A  T  A  C  A  A  A  G  G  G  A  T  A  T  A  T  G  G  C  C  G  A  T  T  A  C  T  A  G
	1	0.59	Novel	T  A  T  A  C  A  A  A  G  G  G  A  T  A  X  A  T  G  G  C  C  G  A  T  T  A  C  X  A  G
	Deviation		03 56 22 15 09 17 03 03 1  05 03 03 05 03 52 04 06 1  06 03 08 02 

In [60]:
show_res('SRR2034356')

Sample SRR2034356
--- K=1------------------------------------------------------------------
	 	    	     	1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
	0	1.0	Novel	C  A  A  T  A  C  A  A  A  G  A  T  X  C  C  G  G  C  T  A  C  T  C  C  C  T  G  C  G  A  G  T  G  T  T  T  G  T  G  T
	Deviation		07 06 51 26 16 17 14 06 06 1  07 1  19 09 05 11 11 05 05 07 05 09 25 06 06 05 05 05 04 04 04 04 04 05 05 06 06 05 21 06
	Objective function: 1.694534530524865
	NSU	: 1.0
	DEV_AVG	: 0.09731042761327388
	DEV_MAX	: 0.5142857142859987
	UNPHASED: 0.08293838862559233
--- K=2------------------------------------------------------------------
	 	    	     	1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
	0	0.17	Novel	C  A  G  A  A  C  A  A  A  G  A  T  T  C  C  G  G  C  T  A  C  T  X  C  C  T  G  C  G  A  G  T  G  T  T  T  G  T  T  T
	1	0.83	Novel	C  A  A  T  A

## Comments.

#### Low abundance STs.
We can see the expected problem of refining the result by adding low abundance STs. So far the terms $NSU$ and $UPR$ do not counterbalance the advantage of optimizing the deviation terms by adding low abundance STs. It would be interesting to evaluate, through well controlled simulation, the impact of this phenomenon.

#### Using phaing to define STs.
So far we consider the phasing only in terms of reads: we want a solution that is in agreement with as many reads as possible. But conversely, especially if we expect a decent coverage, we should expect that the defined STs agree with the reads, i.e. that positions that are close-by are supported by reads. More precisely, a position in an ST is said to be phased if there exists a read that overlaps this positipon and agree with the ST sequence. One can then define a ratio of the number of phased positions over the number of positions (K x number of variable positions) and incorporate this in the objective function. However this is somewhat unsatisfying because a same read can "support" several STs. So this might make this objective term take over the deviation terms. Ideally we would like to assign reads to STs, but this is not do-able due to the large number of reads in the high-coverage data sets: it would generate too many decision variables (already the current ILP requires several Gb of memory).

#### Possible STs.
Currently, all possible combinations of alleles are allowed as possible STs. It would be interesting to use a preprocessing step, using likely a de Bruijn graph, to define a limited set of possible STs (paths in the graph) and enforcing that the inferred STs are among these ones. This turns the problem into a pure LP. But in our MLST context where the 8 loci are independant, there will be a combinatorial explosion of the number of decision variables as it will be the cartesian product of the possible alleles at all loci. But more generally it might be interesting to see the information we can extract from a preliminary assembly of the reads to limit the search space of the (I)LP.

#### Simulations.
The *Borrelia* dataset we have are very different from the datasets obtained by picking STs at random. The number of variable positions is much smaller, which make it amenable to exact optimization. A first set of simulations would be to take the current datasets we have, define STs diversity+sequencing instances that agree with these datasets, together with a controlled level of noise both in the allelic frequency matrix and in the phasing by sequencing reads, and see if we can recover the good solution. It is quite limited (as we simulate within the model we wsolve) but likely needed to validate our approach as a proof of concept.

#### Heuristics.
The problem we aim to solve is quite limited in size (number of variable positions) but the objective function is multi-terms, so it is not immediately clear how it could be attacked using a heuristic, especially as it depends crucially of the abundance assigned to the STs. One could ask if
- it is doable to assign abundances optimally given the STs,
- it is doable to define STs optimally given abundances,
- is-it doable to assign reads optimally given STs.
If that was the case then maybe some kind of smiulated annealing or MCMC exploration could prove to be useful.

General heuristics that might be worth looking at, kernel search: http://cors.sites.olt.ubc.ca/files/2012/11/Abstract-Speranza.pdf

Otherwise, we did not explore the LP relaxation approach followed by rounding, but I am not a specialist of this approach. And while I see that we can relac=x the 0/1 cosntraint on assigning nucleotides to the STs, it is not clear how that would work out for the phasing part of the ILP as it is defined in terms of these 0/1 variables.